In [4]:
class Person:                       
    def __init__(self, name):
        self._name = name

    def getName(self):
        print('fetch...')
        return self._name

    def setName(self, value):
        print('change...')
        self._name = value

    def delName(self):
        print('remove...')
        del self._name
    name = property(getName, setName, delName, "name property docs")

bob = Person('Bob Smith')           
print(bob.name)                     
bob.name = 'Robert Smith'           
print(bob.name)

del bob.name                        
print('-'*20)
sue = Person('Sue Jones')           
print(sue.name)
print(Person.name.__doc__) 

fetch...
Bob Smith
change...
fetch...
Robert Smith
remove...
--------------------
fetch...
Sue Jones
name property docs


In [6]:
class Person:
    def __init__(self, name):
        self._name = name
    @property
    def name(self):                 # name = property(name)            
        "name property docs"
        print('fetch...')
        return self._name

    @name.setter
    def name(self, value):         # name = name.setter(name) 
        print('change...')
        self._name = value

    @name.deleter
    def name(self):                # name = name.deleter(name)     
        print('remove...')
        del self._name


bob = Person('Bob Smith')           
print(bob.name)                     
bob.name = 'Robert Smith'           
print(bob.name)
del bob.name                        
print('-'*20)
sue = Person('Sue Jones')           
print(sue.name)
print(Person.name.__doc__)       

fetch...
Bob Smith
change...
fetch...
Robert Smith
remove...
--------------------
fetch...
Sue Jones
name property docs


In [15]:
# Descriptors provide an alternative way to intercept attribute access

class Name:                             
    "name descriptor docs"

    def __get__(self, instance, owner):
        print('fetch...')
        return instance._name

    def __set__(self, instance, value):
        print('change...')
        instance._name = value

    def __delete__(self, instance):
        print('remove...')
        del instance._name


class Person:                           
    def __init__(self, name):
        self._name = name
    name = Name()  


bob = Person('Bob Smith')               
print(bob.name)                         
bob.name = 'Robert Smith'               
print(bob.name)
del bob.name                            
print('-'*20)
sue = Person('Sue Jones')               
print(sue.name)
print(Name.__doc__) 

fetch...
Bob Smith
change...
fetch...
Robert Smith
remove...
--------------------
fetch...
Sue Jones
name descriptor docs


In [17]:
# Using a nested class
class Person:
    def __init__(self, name):
        self._name = name
    class Name:                                 
        "name descriptor docs"
        def __get__(self, instance, owner):
            print('fetch...')
            return instance._name
        def __set__(self, instance, value):
            print('change...')
            instance._name = value
        def __delete__(self, instance):
            print('remove...')
            del instance._name
    name = Name()

In [21]:
class CardHolder():                      
    acctlen = 8                                
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                       
        self.name = name                       
        self.age  = age                        
        self.addr = addr                       

    def getName(self):
        return self.__name
    
    def setName(self, value):
        value = value.lower().replace(' ', '_')
        self.__name = value
    name = property(getName, setName)
    
    def getAge(self):
        return self.__age
    
    def setAge(self, value):
        if value < 0 or value > 150:
            raise ValueError('invalid age')
        else:
            self.__age = value
    age = property(getAge, setAge)

    def getAcct(self):
        return self.__acct[:-3] + '***'
    def setAcct(self, value):
        value = value.replace('-', '')
        if len(value) != self.acctlen:
            raise TypeError('invald acct number')
        else:
            self.__acct = value
    acct = property(getAcct, setAcct)
    
    def remainGet(self):                       
        return self.retireage - self.age       
    remain = property(remainGet)

In [28]:
# File validate_tester.py
from __future__ import print_function # 2.X

def loadclass():
    import sys, importlib                             
    modulename = sys.argv[1]                          # Module name in command line
    module = importlib.import_module(modulename)      # Import module by name string
    print('[Using: %s]' % module.CardHolder)          # No need for getattr() here
    return module.CardHolder

def printholder(who):
    print(who.acct, who.name, who.age, who.remain, who.addr, sep=' / ')

if __name__ == '__main__':
    # CardHolder = loadclass()
    bob = CardHolder('1234-5678', 'Bob Smith', 40, '123 main st')
    printholder(bob)
    bob.name = 'Bob Q. Smith'
    bob.age  = 50
    bob.acct = '23-45-67-89'
    printholder(bob)

    sue = CardHolder('5678-12-34', 'Sue Jones', 35, '124 main st')
    printholder(sue)
    try:
        sue.age = 200
    except:
        print('Bad age for Sue')

    try:
        sue.remain = 5
    except:
        print("Can't set sue.remain")

    try:
        sue.acct = '1234567'
    except:
        print('Bad acct for Sue')


# py −3 validate_tester.py validate_properties

12345*** / bob_smith / 40 / 19.5 / 123 main st
23456*** / bob_q._smith / 50 / 9.5 / 123 main st
56781*** / sue_jones / 35 / 24.5 / 124 main st
Bad age for Sue
Can't set sue.remain
Bad acct for Sue


In [ ]:
# Using Descriptors to Validate

class CardHolder(object):                        
    acctlen = 8                                  
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                         
        self.name = name                         
        self.age  = age                          
        self.addr = addr   

    class Name(object):
        def __get__(self, instance, owner):      
            return self.name
        def __set__(self, instance, value):
            value = value.lower().replace(' ', '_')
            self.name = value
    name = Name()

    class Age(object):
        def __get__(self, instance, owner):
            return self.age                             
        def __set__(self, instance, value):
            if value < 0 or value > 150:
                raise ValueError('invalid age')
            else:
                self.age = value
    age = Age()

    class Acct(object):
        def __get__(self, instance, owner):
            return self.acct[:-3] + '***'
        def __set__(self, instance, value):
            value = value.replace('-', '')
            if len(value) != instance.acctlen:          
                raise TypeError('invald acct number')
            else:
                self.acct = value
    acct = Acct()
    
    class Remain(object):
        def __get__(self, instance, owner):
            return instance.retireage - instance.age    
        def __set__(self, instance, value):
            raise TypeError('cannot set remain')        
    remain = Remain()